<a href="https://colab.research.google.com/github/DomTHK/DLO_Lab/blob/main/Abgabe/DLO_Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# https://drive.google.com/drive/folders/1hNFbb-Ksc9Qdyx0_CB7tTJFy7-UNwYwy?usp=sharing 

# Bildgröße 300x200 RGB

from google.colab import drive
import torchvision 
from torchvision import transforms

from timeit import default_timer as timer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt

from torchsummary import summary

import torch
import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import Dataset, random_split
from torchvision import datasets, transforms


In [53]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [54]:
# Config Feld
path = f"/content/drive/MyDrive/Colab Notebooks/data"
ModelPath = f'/content/drive/MyDrive/Colab Notebooks/models/pythorch.model'
device = "cuda" if torch.cuda.is_available() else "cpu"
TrainingMode = True

valid_size = 0.1
test_size = 0.2

In [55]:
#  transforms.CenterCrop(224),
transform = transforms.Compose([transforms.Resize((300, 200)),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

dataset = datasets.ImageFolder(path, transform=transform)

In [56]:
# https://discuss.pytorch.org/t/how-to-use-imagefolder-with-list-of-images-for-train-and-test-sets/120247
# trainset = torch.utils.data.Subset(dataset_train, train_indices)
# valset = torch.utils.data.Subset(dataset_val, val_indices)
def get_data(): 
  n_val = int(np.floor(valid_size * len(dataset)))
  n_test = int(np.floor(test_size * len(dataset)))
  n_train = len(dataset) - n_val - n_test

  train_ds, val_ds, test_ds = random_split(dataset, [n_train, n_val, n_test])
  train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)

  valid_dl = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle=True)
  test_dl = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=True)
  return train_dl, test_dl, valid_dl

In [57]:
@torch.no_grad()
def accuracy(x, y, model):
    with torch.no_grad():
        prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [58]:
@torch.no_grad()
def loss(x, y, model):
    prediction = model(x)
    loss = loss_fn(prediction, y)
    return loss.item()

In [59]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 56 * 56, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 3)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 56 * 56) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [60]:
def get_model():
    model = Net().to(device)
    loss_fn = nn.CrossEntropyLoss()  # Kreuzentropie
    optimizer = SGD(model.parameters(), lr=1e-1)
    return model, loss_fn, optimizer

In [61]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [62]:
model, loss_fn, optimizer = get_model()

In [63]:
# 3 - R, G, B 
# 224 - Width 
# 224 - Height
summary(Net(), (3, 224, 224))
# Output Shape
# [-1, 16, 224, 224] 
# First Index: Batch-Size (-1 means it doesn't matter)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             448
              Tanh-2         [-1, 16, 224, 224]               0
         MaxPool2d-3         [-1, 16, 112, 112]               0
            Conv2d-4          [-1, 8, 112, 112]           1,160
              Tanh-5          [-1, 8, 112, 112]               0
         MaxPool2d-6            [-1, 8, 56, 56]               0
            Linear-7                   [-1, 32]         802,848
              Tanh-8                   [-1, 32]               0
            Linear-9                    [-1, 3]              99
Total params: 804,555
Trainable params: 804,555
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 15.50
Params size (MB): 3.07
Estimated Total Size (MB): 19.15
-----------------------------------------

In [64]:
train_dl, test_dl, val_dl = get_data()

In [ ]:
print('Starting training...')

time0 = timer()

epochs = 50

arrPlotX = []
train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []

if TrainingMode:
    for epoch in range(epochs):
        train_epoch_losses, train_epoch_accuracies = [], []

        for ix, batch in enumerate(iter(train_dl)):
            x, y = batch
            
            batch_loss = train_batch(x, y, model, optimizer, loss_fn)  # Mini-Batch
            train_epoch_losses.append(batch_loss)
        train_epoch_loss = np.array(train_epoch_losses).mean()

        for ix, batch in enumerate(iter(train_dl)):
            x, y = batch
            is_correct = accuracy(x, y, model)
            train_epoch_accuracies.extend(is_correct)
        train_epoch_accuracy = np.mean(train_epoch_accuracies)

        val_is_correct = None
        validation_loss = None

        for ix, batch in enumerate(iter(val_dl)):
            x, y = batch
            val_is_correct = accuracy(x, y, model)
            validation_loss = loss(x, y, model)

        val_epoch_accuracy = np.mean(val_is_correct)
        arrPlotX.append(epoch)
        train_losses.append(train_epoch_loss)
        train_accuracies.append(train_epoch_accuracy)
        test_losses.append(validation_loss)
        test_accuracies.append(val_epoch_accuracy)

        # Early stopping
        if val_epoch_accuracy > 0.99:
            break

        print(f"epoch: {epoch}  train_acc: {100 * train_epoch_accuracy:.2f}%  val_acc: {100 * val_epoch_accuracy:.2f}%")

    torch.save(model.state_dict(), ModelPath)

    plt.plot(arrPlotX, train_accuracies)
    plt.plot(arrPlotX, test_accuracies)
    plt.savefig("accuracies.png")
else:
    val_is_correct = None
    for ix, batch in enumerate(iter(test_dl)):
        x, y = batch
        val_is_correct = accuracy(x, y, model)
        validation_loss = loss(x, y, model)

    for ix, var in enumerate(iter(val_is_correct)):
        if var is False:
            print(test_dl.dataset.__getitem__(ix))
    val_epoch_accuracy = np.mean(val_is_correct)
    print(f"test_acc: {100 * val_epoch_accuracy:.2f}%")


Starting training...
